In [247]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pprint
import json
import re

df = pd.read_csv("indeed_data.csv")
# Output File (CSV)
output_data_file = "cleaned_indeed_jobs.csv"

In [248]:
import re
import numpy as np

In [249]:
def extract_years_only(s):
    results = re.findall(r"([0-9]*)\+?.[yY]ear.*", s)
    if results and results[0]:
        return int(results[0])
    else:
        return np.nan

In [250]:
years = df["snippet"].apply(lambda s: extract_years_only(s))
df_years=pd.DataFrame(years.value_counts().sort_index())

In [251]:
df_years=df_years.rename(columns={"snippet":"No. of jobs"})
df_years.index.name ="Required No.of Years of Experience"
df_years

,No. of jobs
Required No.of Years of Experience,
1.0,5
2.0,12
3.0,18
4.0,7
5.0,23
6.0,2
7.0,8
8.0,5
10.0,13


In [252]:
def extract_years(s, tech):
    results = re.findall(r"([0-9]*)\+?.[yY]ear.*[" + tech[0] +  tech[0].upper() + "]" + tech[1:], s)
    result_count = re.findall(tech, s)
    if results and results[0]:
        return int(results[0])
    else:
        return np.nan


In [253]:
def extract_tech_count(s, tech):
    result_count = re.findall(tech, s)

    if result_count:
        return len(result_count)
    else:
        return np.nan

In [254]:
df_tech = pd.DataFrame(columns={"Tech", "No.ofJobs"})
tech_list = ["Python"," R ","SQL","Tableau", "RegEx", "AWS", "Hadoop", "Spark", "Machine Learning", "AI"]

sample = df["snippet"]

for tech in tech_list:
    count =0
    job_tech = sample.apply(lambda s: extract_tech_count(s, tech))
  
    for i in range(0, len(job_tech)):
   
        if(pd.notna(job_tech[i])):
            count += 1
        else:
            continue
    tech_list_count = pd.DataFrame({"Tech":[tech], "No.ofJobs":[count]})
    df_tech = df_tech.append(tech_list_count)

df_tech.set_index("Tech")

,No.ofJobs
Tech,
Python,23
R,5
SQL,22
Tableau,9
RegEx,0
AWS,5
Hadoop,5
Spark,5
Machine Learning,16


In [258]:
def extract_education(s, edu):
    results_masters = re.findall(edu, s)
    if results_masters and results_masters[0]:
        return (results_masters)
    else:
        return np.nan

In [262]:
df_edu = pd.DataFrame(columns={"Edu", "No.ofJobs"})
education_list = [["masters","Masters","MS", "Master's"], ["bachelor's", "Bachelors", "bachelors", "Bachelor's"], ["Phd", "PhD", "Ph.D"]]

for e in education_list:
    count = 0
    for edu in e:
  
        job_edu = sample.apply(lambda s: extract_tech_count(s, edu))

        for i in range(0, len(job_edu)):
            if(pd.notna(job_edu[i])):
                count += 1
            else:
                continue
    edu_list_count = pd.DataFrame({"Edu":[edu], "No.ofJobs":[count]})
    df_edu = df_edu.append(edu_list_count)

df_edu.set_index("Edu")

,No.ofJobs
Edu,
Master's,31
Bachelor's,29
Ph.D,9
